
# Quickstart: Add OPC UA assets to your Azure IoT Operations cluster

<div class="alert alert-block alert-info">
<b>IMPORTANT:</b> Azure IoT Operations Preview – enabled by Azure Arc is currently in PREVIEW.
You shouldn't use this preview software in production environments.

See the [Supplemental Terms of Use for Microsoft Azure Previews](https://azure.microsoft.com/support/legal/preview-supplemental-terms/) for legal terms that apply to Azure features that are in beta, preview, or otherwise not yet released into general availability.</div>

In this quickstart, you manually add OPC UA assets to your Azure IoT Operations Preview cluster. These assets publish messages to the Azure IoT MQ (preview) broker in your Azure IoT Operations cluster. Typically, an OT user completes these steps.

An _asset_ is a physical device or logical entity that represents a device, a machine, a system, or a process. For example, a physical asset could be a pump, a motor, a tank, or a production line. A logical asset that you define can have properties, stream telemetry, or generate events.

_OPC UA servers_ are software applications that communicate with assets. _OPC UA tags_ are data points that OPC UA servers expose. OPC UA tags can provide real-time or historical data about the status, performance, quality, or condition of assets.

In this quickstart, you use will the [Azure CLI to complete some of these tasks](https://learn.microsoft.com/en-us/cli/azure/iot/ops/asset?view=azure-cli-latest).

## Prerequisites

Complete [Quickstart: Deploy Azure IoT Operations to an Arc-enabled Kubernetes cluster](quickstart-virtual-deploy.ipynb) before you begin this quickstart.

To sign in to the Azure IoT Operations portal you need a work or school account in the tenant where you deployed Azure IoT Operations. If you're currently using a Microsoft account (MSA), you need to create a Microsoft Entra ID with at least contributor permissions for the resource group that contains your **Kubernetes - Azure Arc** instance. To learn more, see [Known Issues > Create Entra account](./troubleshoot/known-issues.md#azure-iot-operations-preview-portal).

## What problem will we solve?

The data that OPC UA servers expose can have a complex structure and can be difficult to understand. Azure IoT Operations provides a way to model OPC UA assets as tags, events, and properties. This modeling makes it easier to understand the data and to use it in downstream processes such as the MQ broker and Azure IoT Data Processor (preview) pipelines.

## Sign into the Azure IoT Operations portal

To create asset endpoints, assets and subscribe to OPC UA tags and events, use the Azure IoT Operations (preview) portal. Navigate to the [Azure IoT Operations](https://iotoperations.azure.com) portal in your browser and sign with your Microsoft Entra ID credentials.

<div class="alert alert-block alert-info">
<b>IMPORTANT:</b> You must use a work or school account to sign in to the Azure IoT Operations portal. To learn more, see <a href="./troubleshoot/known-issues.md#azure-iot-operations-preview-portal">Known Issues > Create Entra account</a></div>

## Select your cluster

When you sign in, select **Get started**. The portal displays the list of Kubernetes clusters that you have access to. Select the cluster that you deployed Azure IoT Operations to in the previous quickstart:

![cluster-list.png](media/quickstart-add-assets/cluster-list.png)

<div class="warning" style='background-color:#E9D8FD; color: #69337A; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:left'>
<b>TIP </b>
If you don't see any clusters, you might not be in the right Azure Active Directory tenant. You can change the tenant from the top right menu in the portal. If you still don't see any clusters, that means you are not added to any yet. Reach out to your IT administrator to give you access to the Azure resource group the Kubernetes cluster belongs to from Azure portal. You must be in the <b><i>contributor</i></b> role.</p>
</span>
</div>

## Add an asset endpoint

When you deployed Azure IoT Operations, you chose to include a built-in OPC PLC simulator. In this step, you add an asset endpoint that enables you to connect to the OPC PLC simulator.

To add an asset endpoint:

1. Select **Asset endpoints** and then **Create asset endpoint**:

    ![asset-endpoints.png](media/quickstart-add-assets/asset-endpoints.png)


1. Enter the following endpoint information:

    | Field | Value |
    | --- | --- |
    | Name | `opc-ua-connector-0` |
    | OPC UA Broker URL | `opc.tcp://opcplc-000000:50000` |
    | User authentication | `Anonymous` |
    | Transport authentication | `Do not use transport authentication certificate` |

1. To save the definition, select **Create**.

    This configuration deploys a new asset endpoint called `opc-ua-connector-0` to the cluster. You can use `kubectl` to view the asset endpoints:



In [2]:
kubectl get assetendpointprofile -n azure-iot-operations

NAME                 AGE
opc-ua-connector-0   11m


After you define an asset, an OPC UA connector pod discovers it. The pod uses the asset endpoint that you specify in the asset definition to connect to an OPC UA server. You can use `kubectl` to view the discovery pod that was created when you added the asset endpoint. The pod name looks like `aio-opc-opc.tcp-1-8f96f76-kvdbt`:

In [3]:
kubectl get pods -n azure-iot-operations | grep aio-opc-opc.tcp*

aio-opc-opc.tcp-1-54d78cf79d-vx8pz              2/2     Running   0             12m


When the OPC PLC simulator is running, data flows from the simulator, to the connector, to the OPC UA broker, and finally to the MQ broker.

To enable the asset endpoint to use an untrusted certificate:

<div class="alert alert-block alert-warning">
<b>CAUTION:</b> Don't use untrusted certificates in production environments.
</div>

1. Run the following command on the machine where your cluster is running to apply the configuration to use an untrusted certificate:

In [4]:
kubectl apply -f https://raw.githubusercontent.com/Azure-Samples/explore-iot-operations/main/samples/quickstarts/opc-ua-connector-0.yaml

assetendpointprofile.deviceregistry.microsoft.com/opc-ua-connector-0 configured


The following snippet shows the YAML file that you applied:

```yaml
# Important: do not use in production environments
apiVersion: deviceregistry.microsoft.com/v1beta1
kind: AssetEndpointProfile
metadata:
  name: opc-ua-connector-0
  namespace: azure-iot-operations
spec:
  additionalConfiguration: |-
    {
      "applicationName": "opc-ua-connector-0",
      "defaults": {
        "publishingIntervalMilliseconds": 1000,
        "samplingIntervalMilliseconds": 500,
        "queueSize": 1,
      },
      "session": {
        "timeout": 60000
      },
      "subscription": {
        "maxItems": 1000,
      },
      "security": {
        "autoAcceptUntrustedServerCertificates": true
      }
    }
  targetAddress: opc.tcp://opcplc-000000.azure-iot-operations:50000
  transportAuthentication:
    ownCertificates: []
  userAuthentication:
    mode: Anonymous
  uuid: doe-opc-ua-connector-0
```

1. Find the name of your `aio-opc-supervisor` pod by using the following command:

In [8]:
$AIO_OPC_SUPERVISOR=$(kubectl get pods -n azure-iot-operations | grep aio-opc-supervisor* | awk '{print $1}')

The name of your pod looks like `aio-opc-supervisor-956fbb649-k9ppr`.

1. Restart the `aio-opc-supervisor` pod by using a command that looks like the following example. Use the `aio-opc-supervisor` pod name from the previous step:

In [9]:
kubectl delete pod $AIO_OPC_SUPERVISOR -n azure-iot-operations

pod "aio-opc-supervisor-7b4dd46b45-5qp4n" deleted


In [12]:
kubectl get pods -n azure-iot-operations | grep aio-opc-supervisor*

aio-opc-supervisor-7b4dd46b45-wnxmr             1/1     Running   0          14m


## Manage your assets

After you select your cluster in Azure IoT Operations portal, you see the available list of assets on the **Assets** page. If there are no assets yet, this list is empty:

![create-asset-empty.png](./media/quickstart-add-assets/create-asset-empty.png)

### Create an asset

We will be creating a simulated thermostat asset with the following asset information and OPC UA tags:

| Field | Value |
| --- | --- |
| Asset name | `thermostat` |
| Asset Endpoint | `opc-ua-connector-0` |
| Description | `A simulated thermostat asset` |

<br>

| Node ID            | Tag name    | Observability mode |
| ------------------ | ----------- | ------------------ |
| ns=3;s=FastUInt10  | temperature | none               |
| ns=3;s=FastUInt100 | Tag 10      | none               |



In [16]:
$RESOURCE_GROUP = Read-Host -Prompt "Name of resource group where the Arc-enabled Kubernetes cluster is deployed"
Write-Host "Resource group: $RESOURCE_GROUP" -ForegroundColor Blue

$CLUSTER_NAME = az connectedk8s list -g $RESOURCE_GROUP --query "[0].name" -o tsv

Resource group: rg-emm12122


In [20]:
$NODE_ID_1 = "ns=3;s=FastUInt10"
$TAG_NAME_1 = "temperature"

$NODE_ID_2 = "ns=3;s=FastUInt100"
$TAG_NAME_2 = "Tag 10"

az iot ops asset create `
--name thermostat2 `
-g $RESOURCE_GROUP `
--cluster $CLUSTER_NAME `
--endpoint opc-ua-connector-0 `
--data data_source=$NODE_ID_1 name=$TAG_NAME_1 observability_mode=none sampling_interval=500 queue_size=1 `
--data data_source=$NODE_ID_2 name=$TAG_NAME_2 observability_mode=none sampling_interval=500 queue_size=1

{
  "extendedLocation": {
    "name": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-emm12122/providers/Microsoft.ExtendedLocation/customLocations/emm12122-cl",
    "type": "CustomLocation"
  },
  "id": "/subscriptions/20d5770a-8bb2-402f-9ade-11d91cb478a2/resourceGroups/rg-emm12122/providers/Microsoft.DeviceRegistry/assets/thermostat2",
  "identity": null,
  "kind": null,
  "location": "westus3",
  "managedBy": null,
  "name": "thermostat2",
  "plan": null,
  "properties": {
    "assetEndpointProfileUri": "opc-ua-connector-0",
    "dataPoints": [
      {
        "capabilityId": "temperature",
        "dataPointConfiguration": "{\"samplingInterval\": 500, \"queueSize\": 1}",
        "dataSource": "ns=3;s=FastUInt10",
        "name": "temperature",
        "observabilityMode": "none"
      },
      {
        "capabilityId": "Tag 10",
        "dataPointConfiguration": "{\"samplingInterval\": 500, \"queueSize\": 1}",
        "dataSource": "ns=3;s=FastUInt100",
     

> **Optional**: If you have any additional information for the asset that you want to include such as:
>
> - Manufacturer
> - Manufacturer URI
> - Model
> - Product code
> - Hardware version
> - Software version
> - Serial number
> - Documentation URI
> 
> Use the below cell and replace the values with your own:

In [8]:
$NODE_ID_1 = "ns=3;s=FastUInt10"
$TAG_NAME_1 = "temperature"
$NODE_ID_2 = "ns=3;s=FastUInt100"
$TAG_NAME_2 = "Tag 10"

az iot ops asset create `
--name thermostat2 `
-g rg-aiobugbash1204 `
--cluster aiobugbash1204 `
--endpoint opc-ua-connector-0 `
--data data_source=$NODE_ID_1 name=$TAG_NAME_1 observability_mode=none sampling_interval=500 queue_size=1 `
--data data_source=$NODE_ID_2 name=$TAG_NAME_2 observability_mode=none sampling_interval=500 queue_size=1 `
--manufacturer <MANUFACTURER> `
--manufacturer-uri <MANUFACTURER_URI> `
--model <MODEL> `
--serial-number <SERIAL_NUMBER>

[]



## Verify data is flowing

To verify data is flowing from your assets by using the **mqttui** tool. In this quickstart you run the **mqttui** tool inside your Kubernetes cluster:

1. Run the following command to deploy a pod that includes the **mqttui** and **mosquitto** tools that are useful for interacting with the MQ broker in the cluster:

In [21]:
kubectl apply -f https://raw.githubusercontent.com/Azure-Samples/explore-iot-operations/main/samples/quickstarts/mqtt-client.yaml

pod/mqtt-client created


The following snippet shows the YAML file that you applied:

```yaml
# Important: do not use in production environments
# Creates a pod with mosquitto-clients and mqttui utilities in your cluster
apiVersion: v1
kind: Pod
metadata:
  name: mqtt-client
  # The namespace must match the IoT MQ BrokerListener's namespace
  # Otherwise use the long hostname: aio-mq-dmqtt-frontend.azure-iot-operations.svc.cluster.local
  namespace: azure-iot-operations
spec:
  # Use the "mqtt-client" service account which comes with default deployment
  # Otherwise create it with `kubectl create serviceaccount mqtt-client -n azure-iot-operations`
  serviceAccountName: mqtt-client
  containers:
    # Install mosquitto and mqttui utilities on Alpine linux
  - image: alpine
    name: mqtt-client
    command: ["sh", "-c"]
    args: ["apk add mosquitto-clients mqttui && sleep infinity"]
    volumeMounts:
    - name: mq-sat
      mountPath: /var/run/secrets/tokens
    - name: trust-bundle
      mountPath: /var/run/certs
  volumes:
  - name: mq-sat
    projected:
      sources:
      - serviceAccountToken:
          path: mq-sat
          audience: aio-mq # Must match audience in BrokerAuthentication
          expirationSeconds: 86400
  - name: trust-bundle
    configMap:
      name: aio-ca-trust-bundle-test-only # Default root CA cert
```

<div class="alert alert-block alert-warning">
    <b>CAUTION:</b> This configuration isn't secure. Don't use this configuration in a production environment.
</div>

1. When the **mqtt-client** pod is running, run the following command to create a shell environment in the pod you created:

    ```console
    kubectl exec --stdin --tty mqtt-client -n azure-iot-operations -- sh
    ```

1. At the shell in the **mqtt-client** pod, run the following command to connect to the MQ broker using the **mqttui** tool:

    ```console
    mqttui -b mqtts://aio-mq-dmqtt-frontend:8883 -u '$sat' --password $(cat /var/run/secrets/tokens/mq-sat) --insecure
    ```

1. Verify that the thermostat asset you added is publishing data. You can find the telemetry in the `azure-iot-operations/data` topic.

    ![mqttui-output.png](media/quickstart-add-assets/mqttui-output.png)

    If there's no data flowing, restart the `aio-opc-opc.tcp-1` pod.

    First, find the name of your `aio-opc-opc.tcp-1` pod by using the following command:

In [23]:
$AIO_OPC_OPC=$(kubectl get pods -n azure-iot-operations | grep aio-opc-opc.tcp-1* | awk '{print $1}')

The name of your pod looks like `aio-opc-opc.tcp-1-849dd78866-vhmz6`.

Then restart the `aio-opc-opc.tcp-1` pod by using a command that looks like the following example. Use the `aio-opc-opc.tcp-1` pod name from the previous step:

In [24]:
kubectl delete pod $AIO_OPC_OPC -n azure-iot-operations

pod "aio-opc-opc.tcp-1-54d78cf79d-vx8pz" deleted


The sample tags you added in the previous quickstart generate messages from your asset that look like the following examples:

```json
{
    "Timestamp": "2023-08-10T00:54:58.6572007Z", 
    "MessageType": "ua-deltaframe",
    "payload": {
      "temperature": {
        "SourceTimestamp": "2023-08-10T00:54:58.2543129Z",
        "Value": 7109
      },
      "Tag 10": {
        "SourceTimestamp": "2023-08-10T00:54:58.2543482Z",
        "Value": 7109
      }
    },
    "DataSetWriterName": "oven",
    "SequenceNumber": 4660
}
```

## Discover OPC UA data sources by using Akri

In the previous section, you saw how to add assets manually. You can also use Azure IoT Akri to automatically discover OPC UA data sources and create Akri instance custom resources that represent the discovered devices. Currently, Akri can't detect and create assets that can be ingested into the Azure Device Registry.

When you deploy Azure IoT Operations, the deployment includes the Akri discovery handler pods. To verify these pods are running, run the following command:

In [25]:
kubectl get pods -n azure-iot-operations | grep akri

aio-akri-otel-collector-6744659f9b-746ms        1/1     Running   0          71m
aio-akri-agent-daemonset-4fd47                  1/1     Running   0          71m


In [26]:
kubectl get pods -n azure-iot-operations |  Select-String -Pattern "akri"


aio-akri-otel-collector-6744659f9b-746ms        1/1     Running   0          72m
aio-akri-agent-daemonset-4fd47                  1/1     Running   0          72m



The output from the previous command looks like the following example:

```console
akri-opcua-asset-discovery-daemonset-h47zk     1/1     Running   3 (4h15m ago)    2d23h
aio-akri-otel-collector-5c775f745b-g97qv       1/1     Running   3 (4h15m ago)    2d23h
aio-akri-agent-daemonset-mp6v7                 1/1     Running   3 (4h15m ago)    2d23h
```

On the machine where your Kubernetes cluster is running, run the following command to apply a new configuration for the discovery handler:

In [27]:
kubectl apply -f https://raw.githubusercontent.com/Azure-Samples/explore-iot-operations/main/samples/quickstarts/akri-opcua-asset.yaml

configuration.akri.sh/akri-opcua-asset configured


The following snippet shows the YAML file that you applied:

```yaml
apiVersion: akri.sh/v0
kind: Configuration
metadata:
  name: akri-opcua-asset
  namespace: azure-iot-operations
spec:
  discoveryHandler: 
    name: opcua-asset
    discoveryDetails: "opcuaDiscoveryMethod:\n  - asset:\n      endpointUrl: \"	opc.tcp://opcplc-000000:50000\"\n      useSecurity: false\n      autoAcceptUntrustedCertificates: true\n"
  brokerProperties: {}
  capacity: 1
```

To verify the configuration, run the following command to view the Akri instances that represent the OPC UA data sources discovered by Akri:

In [7]:
kubectl get akrii -n azure-iot-operations

NAME                      CONFIG             SHARED   NODES                          AGE
akri-opcua-asset-dbdef0   akri-opcua-asset   true     ["k3d-k3s-default-server-0"]   2m17s


Note that it may take a few minutes for the instance to show up.

The output from the previous command looks like the following example. You may need to wait for a few seconds for the Akri instance to be created:

```console
NAMESPACE              NAME                      CONFIG             SHARED   NODES            AGE
azure-iot-operations   akri-opcua-asset-dbdef0   akri-opcua-asset   true     ["dom-aio-vm"]   35m
```

Now you can use these resources in the local cluster namespace.

## How did we solve the problem?

In this quickstart, you added an asset endpoint and then defined an asset and tags. The assets and tags model data from the OPC UA server to make the data easier to use in an MQTT broker and other downstream processes. You use the thermostat asset you defined in the next quickstart.

## Clean up resources

If you're not going to continue to use this deployment, delete the Kubernetes cluster that you deployed Azure IoT Operations to and remove the Azure resource group that contains the cluster.

## Next step

[Quickstart: Use Data Processor pipelines to process data from your OPC UA assets](quickstart-process-telemetry.md)